In [4]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from pathlib import Path
import sys, os
sys.path.append(os.path.join(os.path.abspath(""), '..', 'pipeline'))
import sic_matcher as sm

In [2]:
# establishing the file path in such a way to work universally
here = Path().resolve()

In [3]:
# change either code to your desired code
SIC_CODE = "2015" # Poultry Slaughtering and Processing
NAICS_CODE = "311615" # Poultry Processing

In [4]:
FSIS_FILEPATH = "data/fsis-processors-with-location.csv"
FSIS_DATA = here.parent / FSIS_FILEPATH

IG_FILEPATH = "data/2022_Business_Academic_QCQ.txt"
INFOGROUP_2022 = [here.parent / IG_FILEPATH]

## Method Using Dask DataFrames

In [5]:
# def dask_sic_matches_df(mst_df, sic_code):
    
#     """
#         function will filter the master dataframe input that contains everything read in from csv file.
#         This will filter the master dataframe to contain only those rows that contain the SIC Code that
#         is input into the function
        
#         Args:
#             mst_df (dataframe): master dataframe resulted from reading in the csv file
#             sic_code  (string): SIC Code that the user 
            
#         Returns:
#             a dataframe that contains only the rows where the input SIC Code is listed for that row (busienss)
#     """
    
#     # reassure SIC Code is a string    
#     sic_code = str(sic_code)
    
#     # this filters checks SIC Code 1 through 4 & Primary SIC Code if they contain desired SIC Code we are searching for
#     filtered_df = mst_df[ mst_df['SIC CODE'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 1'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 2'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 3'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 4'].str.contains(sic_code, na=False) |
#                           mst_df['PRIMARY SIC CODE'].str.contains(sic_code, na=False)
#                          ]
    
#     result = filtered_df.compute()
            
#     return result

In [5]:
# Read in csv file using Dask dataframe (rather than pandas)
df = dd.read_csv(INFOGROUP_2022[0], dtype=str, encoding='unicode_escape')
df.columns = df.columns.str.upper() # force all columns to be uppercase
df.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MIRAGE DRAPERIES,4731 E SUNNY DUNES RD,PALM SPRINGS,CA,92264,1536,065,760,2,A,...,6,044804,1,033.812119,-116.494500,0,40140,2,348,06065
1,NaN,11700 OLIO RD,FISHERS,IN,46037,7618,057,317,1,A,...,7,110808,1,039.959999,-085.921268,P,26900,2,294,18057
2,STONE RESIN SURFACING LLC,1 MAYFLOWER PL,MILFORD,CT,06460,4520,009,203,2,D,...,7,150200,2,041.207739,-073.070136,P,35300,2,408,09009
3,NaN,2121 7TH ST,PARKERSBURG,WV,26101,3803,107,740,1,NaN,...,7,000702,1,039.269405,-081.520599,P,37620,2,425,54107
4,IRONWOOD STATE PRISON,19005 WILEYS WELL RD,BLYTHE,CA,92225,2287,065,760,2,A,...,7,046900,1,033.570065,-114.898031,0,40140,2,348,06065


In [6]:
selected_code = sm.dask_sic_matches_df(df, SIC_CODE)

In [31]:
FILEPATH = "data/code_" + SIC_CODE + ".csv"
selected_code.to_csv(here.parent / FILEPATH) # save as CSV file

## Panda Dataframe Method

In [ ]:
# read in entire Infrogroup 2022 file
df = pd.read_csv(INFOGROUP_2022[0], dtype=str)

In [7]:
df_sic = sm.sic_matches_df(df, '0291')

In [ ]:
df_sic.head()

In [7]:
# to convert your dataframe to a csv
FILENAME = "../data/code_" + SIC_CODE + ".csv"
df_sic.to_csv(here.parent / FILENAME)

## New Dataframe containing only certain states (using dask methodology)

In [5]:
certain_states = ['IA', 'OK', 'MO', 'OK', 'AL', 'LA', 'MS', 'IL', 'IN', 'OH', 'KY', 'TN', 'AR', 'NC', 'SC', 'GA']

In [ ]:
df = dd.read_csv(INFOGROUP_2022[0], dtype=str, encoding='unicode_escape')
df.columns = df.columns.str.upper() # force all columns to be uppercase
df.head()

In [7]:
df_certain_states = df[df['STATE'].isin(certain_states)]

In [8]:
df_certain_states = df_certain_states.compute()

In [9]:
len(df_certain_states)

4651574

In [ ]:
df_certain_states['STATE'].value_counts()

In [11]:
df_certain_states.to_csv(here.parent / "data/certain_states.csv") # save as CSV file